In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
from networkx.generators.community import stochastic_block_model
from networkx.linalg.graphmatrix import adjacency_matrix
from numpy.linalg import norm
from scipy.linalg import eigh
from matplotlib import cm 
import time
# set random seeds
# random.seed(11)
# np.random.seed(11)

In [ ]:
def LP_MCAP(C, n, K):
    
    C = C.mean(axis=1).reshape(-1, 1) - C
    G = C.mean(axis=0)
    mu = np.zeros(K)
    m = n // K
    
    for k in range(K):
        diff_C_column = C[:, k:k+1] - C if k == 0 else np.hstack((diff_C_column, C[:, k:k+1]- C))
     
    set_M_label = np.argmin(C[0:m, :] - G, axis=1)
    P = np.zeros((K**2, K))
    P_inx = np.zeros(K**2)
    P_i_j = np.zeros(K**2)
    for k in range(K):
        tilde_Si = np.where(set_M_label == k)[0]
        mu[k] = np.size(tilde_Si)
        if mu[k] > 0:
            diff_i_j = diff_C_column[tilde_Si, k*K:(k+1)*K]
            P_i_j[k*K:(k+1)*K] = np.max(diff_i_j, axis=0)
            inx_p_i_j = np.argmax(diff_i_j, axis=0)
            P[k*K:(k+1)*K, :] = C[tilde_Si[inx_p_i_j], :]
            P_inx[k*K:(k+1)*K] = tilde_Si[inx_p_i_j]
            
    
    label_vector = np.zeros(n)
    label_vector[:m] = set_M_label
    K_set = np.array([i for i in range(K)])
    
    for i in range(m, n):
        a = np.argmin(C[i, :]- G)
        label_vector[i] = a
        mu[a] += 1
        
        set_M_label = label_vector[:i+1]
        
        tilde_Si = np.where(set_M_label == a)[0]
        if mu[a] > 0:
            diff_i_j = diff_C_column[tilde_Si, a*K:(a+1)*K]
            P_i_j[a*K:(a+1)*K] = np.max(diff_i_j, axis=0)
            inx_p_i_j = np.argmax(diff_i_j, axis=0)
            P[a*K:(a+1)*K, :] = C[tilde_Si[inx_p_i_j], :]
            P_inx[a*K:(a+1)*K] = tilde_Si[inx_p_i_j]
            
            
        if mu[a] > m:
            inx_J = np.array([a])
            num_boundary = 0
            boundary_inx, boundary_s_h = None, None
            
            while True:
                e_J = np.zeros(K)
                e_J[inx_J] = 1
                X_J = e_J - e_J.mean() 
                inx_K_J = np.array([tmp for tmp in K_set if tmp not in inx_J])
                
                size_J = np.size(inx_J)
                size_K_J = np.size(inx_K_J)
                g_i_j = np.zeros((size_J*size_K_J, K))
                diff = np.zeros(size_J*size_K_J)
                set_g_i_j = np.zeros((size_J*size_K_J, 2))
                
                k = 0
                for l in inx_J:
                    for j in inx_K_J:
                        diff[k] = P_i_j[l*K+j]-(G[l]-G[j])
                        set_g_i_j[k, :] = np.array([l,j])
                        g_i_j[k, :] = G + diff[k]*X_J
                        k += 1
                
                inx = np.argmax(diff)
                G = g_i_j[inx, :]
                s = int(set_g_i_j[inx, 0])
                h = int(set_g_i_j[inx, 1])
                p_s_h = P[s*K+h, :]
                inx_p_s_h = int(P_inx[s*K+h])
                
                if mu[h] < m:
                    
                    label_vector[inx_p_s_h] = h
                    mu[h] += 1
                    mu[s] -= 1
                    
                    if num_boundary > 0 and (mu > m).sum() > 0:
                        T = 1
                        inx_a = np.where(boundary_s_h[:, 0] == a)[0]
                        inx_collect = dict()
                        inx_collect[T] = inx_a.copy()
                        while True:
                            T += 1
                            inx_T = None
                            for k in inx_a:
                                tmp = np.where(boundary_s_h[:,0] ==boundary_s_h[k, 1])[0]
                                inx_T = tmp if inx_T is None else np.hstack((inx_T, tmp))
                            inx_collect[T] = inx_T.copy()
                            label_vectors = np.zeros((T*n, T))
                            mus = np.zeros((T*K, T))
                            label_vectors[:n,0] = label_vector
                            mus[:K, 0] = mu
                            
                            for t in range(T):
                                t1 = 0
                                for l in inx_collect[t+1]:
                                    [mu2, label_vector2] = label_exchange(
                                        label_vectors[t*n:(t+1)*n, :].flatten(order='F'),
                                        mus[t*K:(t+1)*K, :].flatten(order='F'),
                                        boundary_inx,
                                        boundary_s_h,
                                        l,
                                        K,
                                        inx_collect,
                                        t,
                                        t1
                                    )
                                    if (mu2 <= m).sum() == K:
                                        mu = mu2.copy()
                                        label_vector = label_vector2.copy()
                                        break
                                    else:
                                        label_vectors[(t+1)*n:(t+2)*n,t1] = label_vector2
                                        mus[(t+1)*K:(t+2)*K, t1] = mu2
                                        t1 += 1
                                    
                                if (mu <= m).sum() == K:
                                    break
                                    
                            if (mu <= m).sum() == K:
                                break
                            else:
                                inx_a = inx_collect[T].copy()
                            
                            
                    break
                
                else:
                    inx_J = np.hstack((inx_J, h))
                    num_boundary += 1
                    if num_boundary == 1:
                        boundary_inx = np.array([inx_p_s_h])
                        boundary_s_h = np.array([[s, h]])
                    else:
                        boundary_inx = np.hstack((boundary_inx, inx_p_s_h))
                        boundary_s_h = np.vstack((boundary_s_h, np.array([s, h])))
                        
                
            set_M_label = label_vector[:i+1]
            for k in range(K):
                tilde_Si = np.where(set_M_label == k)[0]
                if mu[k] > 0:
                    diff_i_j = diff_C_column[tilde_Si, a*K:(a+1)*K]
                    P_i_j[a*K:(a+1)*K] = np.max(diff_i_j, axis=0)
                    inx_p_i_j = np.argmax(diff_i_j, axis=0)
                    P[a*K:(a+1)*K, :] = C[tilde_Si[inx_p_i_j], :]
                    P_inx[a*K:(a+1)*K] = tilde_Si[inx_p_i_j]
    
    H = np.zeros((n, K))
    for k in range(K):
        H[label_vector==k, k] = 1
                
                   
    return H

In [ ]:
def label_exchange(l_v, mu, boundary_inx, boundary_s_h, l, K, inx_collect, T, T1):
    k = np.size(mu) // K
    n = np.size(l_v) // k
    m = n // K
    flag = 0
    
    mu_s, l_v_s = dict(), dict()
    
    for t in range(k):
        mu1 = mu[t*K:(t+1)*K]
        l_v1 = l_v[t*n:(t+1)*n]
        inx = boundary_inx[l]
        s = boundary_s_h[l, 0]
        h = boundary_s_h[l, 1]
        mu1[s] -= 1
        l_v1[inx] = h
        mu1[h] += 1
        
        if (mu1 <= m).sum() == K:
            mu = mu1.copy()
            l_v = l_v1.copy()
            break
        else:
            flag += 1
            
        mu_s[t] = mu1.copy()
        l_v_s[t] = l_v1.copy()
    
    if flag == k:
        if T > 0:
            s_h_T = boundary_s_h[inx_collect[T+1][T1], :]
            for t in range(k):
                s_h = boundary_s_h[inx_collect[T][t], :]
                if s_h[1] == s_h_T[0]:
                    break
        mu = mu_s[t].copy()
        l_v = l_v_s[t].copy()
    
    
    return mu, l_v

In [ ]:
def spectral_init(As, n, K):
    
    H = np.zeros((n, K))
    mu = 0.5
    r = mu * np.sqrt(K / n)
    
    _, eigen_vec = eigh(As, subset_by_index=[n - K, n - 1])
    M = (eigen_vec @ eigen_vec.T).real
    M1 = np.tile(np.diag(M).reshape(-1,1), (1, n))
    dist = np.sqrt(M1 + M1.T - 2 * M)
    Q = dist < r
    S = np.array([i for i in range(n)])
    
    C = dict()
    
    for k in range(K):
        Q_s = Q[S, :]
        num_ele = (Q_s != 0).sum(axis=1)
        t = np.argmax(num_ele)
        
        T = np.where(Q[S[t], :])[0]
        C[k] = T
        H[T, k] = 1
        
        S = np.array([tmp for tmp in S if tmp not in T])
        
    if np.size(S) > 0:
        for i in S:
            dist_i = np.zeros(K)
            for k in range(K):
                T = C[k]
                dist_i[k] = np.sum(dist[i, T]) / np.size(T)
            inx_k = np.argmin(dist_i)
            H[i, inx_k] = 1
    
    
    return H

In [ ]:
class GPM_SF_ContAng:
    
    def __init__(self, size_of_clusters, probs_of_sbm, num_of_clusters):

        self.graph = np.asarray(adjacency_matrix(stochastic_block_model(size_of_clusters, probs_of_sbm)).todense())
        self.true_angles_by_cluster = dict()
        self.size_of_clusters = size_of_clusters
        self.N = size_of_clusters.sum()
        self.num_of_clusters = num_of_clusters
        self.alpha_struc = None
        self.A_k = None
        self.Phi = None
        self.Pi = np.eye(self.N)
        self.Rf = None
        self.cluster_id = None
        self.estimated_angles = None
    
    def generate_angles(self):

        self.alpha_struc = np.random.uniform(low=0, high=2*np.pi, size=(self.N, self.N))
        self.alpha_struc[np.tril_indices(self.N)] = 0
        self.alpha_struc = self.alpha_struc + np.mod(-self.alpha_struc.T, 2*np.pi)
        start_idx, end_idx = 0, 0
        for idx, b_size in enumerate(self.size_of_clusters):
            end_idx += b_size
            alphas = np.random.uniform(low=0, high=2*np.pi, size=b_size)
            self.true_angles_by_cluster.setdefault(idx, alphas)
            self.alpha_struc[start_idx:end_idx, start_idx:end_idx] = np.mod(
                alphas.reshape(-1, 1) - alphas,
                2*np.pi
            )
            start_idx += b_size

        return None
    
    def construct_A_k(self):

        self.A_k = self.graph * np.exp(1j * self.alpha_struc)
        _, eigen_vec = eigh(
            self.A_k,
            subset_by_index=[self.N - self.num_of_clusters, self.N - 1]
        )
        self.Phi = eigen_vec.T

        return None
    
    def initialization(self):
        
        H = spectral_init(self.graph, self.N, self.num_of_clusters)
        H = LP_MCAP(H, self.N, self.num_of_clusters)
        U = self.Phi.T
        V = np.zeros((self.N, self.num_of_clusters), dtype='complex_')
        for m in range(self.num_of_clusters):
            cul_m = np.where(H[:,m] == 1)[0]
            pivot = np.random.choice(cul_m)
            U_p = U[pivot,:]
            V[cul_m, m] = np.exp(1j * np.angle((U[cul_m,:]*np.conjugate(U_p)).sum(axis=1)))
            
        return V
    
    
    def GPM_iterations(self, max_iter=50):
        
        V = self.initialization()    
        for iter in range(max_iter):
            
            AV = self.A_k @ V
            H = LP_MCAP(np.abs(AV), self.N, self.num_of_clusters)
            V = np.zeros((self.N, self.num_of_clusters), dtype='complex_')
            for m in range(self.num_of_clusters):
                cul_m = np.where(H[:,m] == 1)[0]
                V[cul_m, m] = np.exp(1j * np.angle(AV[cul_m, m]))
                                     
        self.cluster_id = np.argmax(H, axis=1)
        self.estimated_angles = np.angle(V[np.array(range(self.N)), self.cluster_id])
        
        return None
                                     
                        

In [ ]:
n_sample = 20
b_sizes = np.array([500, 500])
n = b_sizes.sum()
M = 2
success_rate = np.zeros((41, 16))
angle_error = np.zeros((41, 16))

In [ ]:
for i in range(2, 16):
    for j in range(2, 41):
        start = time.perf_counter()
        for t in range(n_sample):
            p = i * np.log(n) / n
            q = j * np.log(n) / n
            sbm_probs = q * np.ones((M, M)) + (p - q) * np.eye(M)
            gpm = GPM_SF_ContAng(b_sizes, sbm_probs, M)
            gpm.generate_angles()
            gpm.construct_A_k()
            gpm.GPM_iterations()
            if (gpm.cluster_id[:500].sum() == 0 and gpm.cluster_id[500:].sum() == 500) \
                    or (gpm.cluster_id[:500].sum() == 500 and gpm.cluster_id[500:].sum() == 0):
                success_rate[j, i] += 1

            angle_error_cluster = np.zeros(M)
            for m in range(M):
                tmp1 = np.exp(-1j*gpm.estimated_angles[m * 500:(m + 1) * 500])
                tmp2 = np.exp(1j*gpm.true_angles_by_cluster[m])
                d_ang = np.angle((tmp1*tmp2).mean())
                angle_diff = np.abs(np.mod(gpm.estimated_angles[m * 500:(m + 1) * 500] + d_ang, 2*np.pi)
                                    - gpm.true_angles_by_cluster[m])
                angle_error_cluster[m] = np.min(np.vstack((angle_diff, 2*np.pi - angle_diff)), axis=0).max()
                
#                 angle_estimate = gpm.estimated_angles[m * 500:(m + 1) * 500]
#                 angle_true = gpm.true_angles_by_cluster[m]
#                 tmp = np.zeros(360)
#                 for freq in range(360):
#                     angle_diff = np.abs(np.mod(angle_estimate + freq*np.pi/180, 2*np.pi) - angle_true)
#                     tmp[freq] = np.min(np.vstack((angle_diff, 2*np.pi - angle_diff)), axis=0).max()
                    
    
#                 angle_error_cluster[m] = np.min(tmp) 
                
            angle_error[j, i] += np.max(angle_error_cluster)

        print(i, j, angle_error[j, i] / n_sample, success_rate[j, i] / n_sample)
        print("cost %s second" % (time.perf_counter() - start))

In [ ]:
plt.figure()
plt.imshow(success_rate[2:41, 2:16][::-1, :] / n_sample, aspect='0.3', extent=[2, 15, 2, 40], cmap='gray')
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
plt.colorbar()
plt.grid(linestyle = '--')
plt.savefig("exact_recovery_GPM_SF_ContAng")

In [ ]:
initial_cmap = cm.get_cmap('gray')
reversed_cmap=initial_cmap.reversed()
plt.figure()
plt.imshow(angle_error[2:41, 2:16][::-1, :] / n_sample, aspect='0.3', extent=[2, 15, 2, 40], cmap=reversed_cmap)
plt.xlabel(r'$\alpha$')
plt.ylabel(r'$\beta$')
plt.colorbar()
plt.grid(linestyle = '--')
plt.savefig("angle_recovery_GPM_SF_ContAng")